# Data Cleaning and Processing

Loading in the InsideAirbnb data and cleaning it up for our later model.

This includes removing unnecessary columns, removing outliers, and accounting for multicollinearity.

In [121]:
# Load libraries
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from src import drop_column_using_vif_, show_vif_values

from statsmodels.stats.outliers_influence import variance_inflation_factor 
import imageio

## Loading in the Data

Here we also removed unnecessary columns.

In [122]:
# Open yvr_listing_data.csv in the data folder
listings_df = pd.read_csv(os.path.join('data', 'yvr_listing_data.csv'))

In [123]:
# Open yvr_listing_data.csv in the data folder
listings_df = pd.read_csv(os.path.join('data', 'yvr_listing_data.csv'))

# Exclude columns manually that are completly textual description or apparently non-related to legality(including coordinates).
# Also exclude some redundant variables like 'neighbourhood' and 'neighbourhood_cleansed'

excluded_columns = ['listing_url','scrape_id', 'last_scraped', 'source', 
                       'name','description', 'neighborhood_overview', 'picture_url', 
                       'host_id', 'host_url', 'host_name', 'host_since', 
                       'host_location', 'host_about', 'host_thumbnail_url', 
                       'host_picture_url', 'latitude', 'longitude', 'calendar_updated', 
                       'calendar_last_scraped', 'amenities', 'bathrooms_text',
                       'first_review','last_review','neighbourhood','property_type','host_neighbourhood',
                       'maximum_minimum_nights','maximum_nights','minimum_minimum_nights',
                       'maximum_maximum_nights','minimum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm']

remained_columns = [col for col in listings_df if col not in excluded_columns]
remained_columns = list(set(remained_columns))

# Delete all textual description columns 

listings_df = listings_df[remained_columns]

In [124]:
# Dropped completely empty columns
listings_df= listings_df.dropna(axis=1, how='all')

## Finding "Legal" Listings

Using regex, we scan through the listings licenses and determine which ones are valid.

In [125]:
%%capture --no-stdout
"""
Create a new column titled "legal_listing" that contains the boolean describing whether or not the listing has a valid license.
The column is True if the listing has a valid license or does not require one and False if the listing does not have a valid license.
To compute the value of the column, we use the following logic:

If the listing has a number in the "license" column with the regex pattern of r'.*?(\d{2}[-\s]?\d{3}[-\s]?\d{3}).*?' 
OR the listing has a number in the "minimum_nights" column with a value equal to or greater than 30,
THEN the "legal_listing" is True. ELSE the "valid_license" is False.

Note:
The regex pattern '.*?(\d{2}[-\s]?\d{3}[-\s]?\d{3}).*?' is used to find a numbers with the pattern ##-###### or ##-###-### with 
spaces/dashes/nothing in between the numbers. The number can be surrounded by any number of characters. 
TODO: Verify this is the correct pattern for the license numbers and find any other ways of verifying legitimate license numbers.
"""

###Just found there are some values like 'dd-ddd-ddd', so I changed regex pattern for better compatibility
#regex_pattern = re.compile(r'.*?(\d{2}[-\s]?\d{6}).*?')
regex_pattern = re.compile(r'.*?(\d{2}[-\s]?\d{3}[-\s]?\d{3}).*?')

# Create the valid_license column using the logic described above
listings_df['legal_listing'] = listings_df['license'].str.contains(regex_pattern) 

# Create new dataframe storing values after normalization or preprocessing
listings_df_cleaned = pd.DataFrame()
listings_df_cleaned['id'] = listings_df['id']
listings_df_cleaned['legal_listing'] = listings_df['legal_listing']

# Drop the 'license' column for better processing
listings_df.drop('license',axis=1, inplace=True)

# Print count of valid and invalid licenses
print(listings_df['legal_listing'].value_counts(dropna=False))

legal_listing
True     4500
NaN      1800
False     395
Name: count, dtype: int64


In [126]:
listings_df['legal_listing'] = listings_df['legal_listing'].fillna(False)

In [127]:
# Drop listings with 'minimum_nights > 30' based on the regulation in Vancouver
listings_df = listings_df[listings_df['minimum_nights']<30]

In [128]:
# Print count of valid and invalid licenses
print(listings_df['legal_listing'].value_counts(dropna=False))

legal_listing
True     4001
False     299
Name: count, dtype: int64


## Dealing with Data Types

- Converting variables to the correct data types while also cleaning unnecessary characters.
- Accounting for categorical data with one-hot encoding.

In [129]:
#print(listings_df.columns)
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4300 entries, 0 to 6694
Data columns (total 39 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_acceptance_rate                          3756 non-null   object 
 1   number_of_reviews                             4300 non-null   int64  
 2   number_of_reviews_l30d                        4300 non-null   int64  
 3   instant_bookable                              4300 non-null   object 
 4   minimum_nights                                4300 non-null   int64  
 5   host_has_profile_pic                          4300 non-null   object 
 6   bedrooms                                      3401 non-null   float64
 7   host_total_listings_count                     4300 non-null   int64  
 8   availability_90                               4300 non-null   int64  
 9   room_type                                     4300 non-null   object

**DROP columns with low varience**

In [130]:
# Select every columns with dtype float or int
# Eliminate the impact that one-hot columns' varience is low
listings_df_num = listings_df.select_dtypes(include=['float64','int64'])

# Calculate the varience
variances = listings_df_num.var()

In [131]:
# Set the bottom boundary 
low_threshold = 0.1  
high_threshold = 1000000  
# Drop columns with low varience
low_variance_cols = variances[variances < low_threshold].index
high_variance_cols = variances[variances > high_threshold].index
listings_df.drop(low_variance_cols, axis=1, inplace=True)
listings_df.drop(high_variance_cols, axis=1, inplace=True)

### Dealing with Object Columns

In [132]:
# Print names of object columns
print(listings_df.select_dtypes(include=['object']).columns)

Index(['host_acceptance_rate', 'instant_bookable', 'host_has_profile_pic',
       'room_type', 'host_response_rate', 'neighbourhood_cleansed',
       'has_availability', 'price', 'host_response_time', 'host_verifications',
       'host_identity_verified', 'host_is_superhost'],
      dtype='object')


In [133]:
print(listings_df['legal_listing'].value_counts())

legal_listing
True     4001
False     299
Name: count, dtype: int64


In [134]:
#converting 'price' column
# Convert price to a float variable
if listings_df['price'].dtype == 'object':
    listings_df['price'] = listings_df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# Convert 'host_acceptance_rate' to a float variable
if listings_df['host_acceptance_rate'].dtype == 'object':
    listings_df['host_acceptance_rate'] = listings_df['host_acceptance_rate'].str.replace('%', '').astype(float)

# Convert 'host_response_time' to a float variable
# The reason is a bit far-fetched for range(0,0.25,0.5,0.75,1), just make it easier for regression model operating. 
# Moreover it does make sense, to some extent
if listings_df['host_response_time'].dtype == 'object':
    listings_df['host_response_time'] = listings_df['host_response_time'].map({
        'within an hour': 1, 'within a few hours': 0.75, 'within a day': 0.5, 'a few days or more': 0.25}).fillna(0)

# Convert 'host_response_rate' to a float variable
if listings_df['host_response_rate'].dtype == 'object':
    listings_df['host_response_rate'] = listings_df['host_response_rate'].str.replace('%', '').astype(float)

# Convert 'host_verifications' to a float variable
if listings_df['host_verifications'].dtype == 'object':
    listings_df['host_verifications'] = listings_df['host_verifications'].map({
        "['email', 'phone', 'photographer', 'work_email']": 1, "['email', 'phone', 'work_email']": 0.75, 
        "['email', 'phone']": 0.5, "['phone', 'work_email']":0.5, 
        "['phone']": 0.25, "['email']": 0.25}).fillna(0)


# Convert 'host_is_superhost' to a bool variable
if listings_df['host_is_superhost'].dtype == 'object':
    listings_df['host_is_superhost'] = listings_df['host_is_superhost'].map({'t': 1, 'f': 0})

# Convert 'host_has_profile_pic' to a bool variable
if listings_df['host_has_profile_pic'].dtype == 'object':
    listings_df['host_has_profile_pic'] = listings_df['host_has_profile_pic'].map({'t': 1, 'f': 0})

# Convert 'has_availability' to a bool variable
if listings_df['has_availability'].dtype == 'object':
    listings_df['has_availability'] = listings_df['has_availability'].map({'t': 1, 'f': 0})

# Convert 'instant_bookable' to a bool variable
if listings_df['instant_bookable'].dtype == 'object':
    listings_df['instant_bookable'] = listings_df['instant_bookable'].map({'t': 1, 'f': 0})

# Convert 'host_identity_verified' to a bool variable
if listings_df['host_identity_verified'].dtype == 'object':
    listings_df['host_identity_verified'] = listings_df['host_identity_verified'].map({'t': 1, 'f': 0})

In [135]:
# Check the object columns again
object_columns = listings_df.select_dtypes(include='object')

object_columns_name = list(object_columns.columns)
object_columns_name


['room_type', 'neighbourhood_cleansed']

In [136]:
listings_df['host_verifications'].unique()

array([0.5 , 0.25, 0.75])

### One-hot to code categorical columns

In [137]:
print("Dropped categories:")
for colname in object_columns_name:
    # convert room_type column to 'category' dtype
    listings_df[colname] = listings_df[colname].astype('category')

    # Since we will be dropping the first category of each column, 
    # lets print out the first category of each column so we know what we are dropping
    print(colname, ':', listings_df[colname].cat.categories[0])

    # applying one-hot coding (drop_first means eliminate one freedom degree to prevent multicollinearity)
    one_hot_encoded = pd.get_dummies(listings_df[colname], prefix=colname, drop_first=True)
    # join new columns back to DataFrame
    listings_df = listings_df.join(one_hot_encoded)

Dropped categories:
room_type : Entire home/apt
neighbourhood_cleansed : Arbutus Ridge


In [138]:
# Print types of all columns
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4300 entries, 0 to 6694
Data columns (total 59 columns):
 #   Column                                           Non-Null Count  Dtype   
---  ------                                           --------------  -----   
 0   host_acceptance_rate                             3756 non-null   float64 
 1   number_of_reviews                                4300 non-null   int64   
 2   number_of_reviews_l30d                           4300 non-null   int64   
 3   instant_bookable                                 4300 non-null   int64   
 4   minimum_nights                                   4300 non-null   int64   
 5   host_has_profile_pic                             4300 non-null   int64   
 6   bedrooms                                         3401 non-null   float64 
 7   host_total_listings_count                        4300 non-null   int64   
 8   availability_90                                  4300 non-null   int64   
 9   room_type               

## Preparing for VIF Analysis

In [139]:
listings_df_VIF = listings_df.select_dtypes(include=['bool','float64','int64'])
listings_df_VIF = listings_df_VIF.astype('float64')

**Using VIF to filter relating variables**

In [140]:
# calculating VIF

# Drop all rows containing NAs or infs in listings_df_VIF
listings_df_VIF.replace([np.inf, -np.inf], np.nan, inplace=True)
listings_df_VIF.dropna(inplace=True)

In [141]:
print(listings_df_VIF['legal_listing'].value_counts())

legal_listing
1.0    2489
0.0     110
Name: count, dtype: int64


In [142]:
print(listings_df['legal_listing'].value_counts())

legal_listing
True     4001
False     299
Name: count, dtype: int64


In [143]:


def drop_column_using_vif_(df, thresh=5):
    '''
    This function is adjusted from: https://stackoverflow.com/a/51329496/4667568

    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: (default 5) the threshould VIF value. If the VIF of a variable is greater than thresh, it should be removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    vif_history = []  # save the list for VIFs of each iteration

    while True:
        
        # adding a constatnt item to the data. add_constant is a function from statsmodels (see the import above)
        df_with_const = add_constant(df,has_constant='add')

        if 'const' in df_with_const.columns:
            vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])], name= "VIF", 
                                index=df_with_const.columns).to_frame()

            # drop the const
            vif_df = vif_df.drop('const')
            vif_history.append(vif_df.copy()) 
        else:
            raise ValueError("constant column 'const' not successfully added")
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        # If there are multiple variabels with VIF>thresh, only one of them is removed. This is because we want to keep as many variables as possible
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {} (VIF: {})'.format(index_to_drop, vif_df.loc[index_to_drop, 'VIF']))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df, vif_history


def show_vif_values(df, dependent_variable):
    """
    Takes a dataframe and the name of the dependent variable, 
    and returns a dataframe with the VIF values for each column (independent variables).
    """
    # Exclude the dependent_variable column from the analysis
    df.drop(columns=dependent_variable, inplace=True)

    df_with_const = add_constant(df,has_constant='add')

    vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])], name= "VIF", 
                            index=df_with_const.columns).to_frame()

    vif_df = vif_df.drop('const')

    # Sort the dataframe by VIF values in descending order
    vif_df = vif_df.sort_values(by='VIF', ascending=False)

    return vif_df

## VIF Filtering

In [144]:
def make_vif_gif(vif_history, num_interpolations=5, keyframe_repeats=5, interpolated_frame_repeats=1, gif_filename='vif_evolution.gif'):
    filenames = []

    for i, vif_df in enumerate(vif_history):
        # sort the column as ascending order
        vif_df = vif_df.sort_values(by='VIF')

        plt.figure(figsize=(10, 6))
        bars = plt.bar(vif_df.index, vif_df['VIF'], color='skyblue')
        plt.title(f'VIF process')
        plt.xticks(rotation=45, ha="right", fontsize=5)
        plt.ylabel('VIF')

        # add the label above the bar
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2.0, yval, round(yval, 1), va='bottom',ha='center', fontsize=8)
        # add the horizontal line as Y=2
        plt.axhline(y=2, color='red', linestyle='--', linewidth=1)
        # get the max of X axis，and add the label 
        xmax = max(bar.get_x() + bar.get_width() for bar in bars)
        plt.text(xmax, 2, ' VIF=2', va='center', ha='left', color='gray',fontsize=10)

        plt.tight_layout()

        # save to png
        filename = f'vif_{i}.png'
        plt.savefig(filename)
        plt.close()
        filenames.append(filename)

        # iterations numbers
        repeats = keyframe_repeats if i % (num_interpolations + 1) == 0 else interpolated_frame_repeats
        for _ in range(repeats - 1):  
            filenames.append(filename)

    # generate GIF
    with imageio.get_writer(gif_filename, mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    
    # delete all those pictures
    for filename in set(filenames):
        os.remove(filename)


In [145]:
def linear_interpolate_vif(vif_history, num_interpolations=10):

    interpolated_vif = []

    for i in range(len(vif_history) - 1):
        current_frame = vif_history[i]
        next_frame = vif_history[i + 1]

        # make sure columns share same sequence
        next_frame = next_frame.reindex(current_frame.index)

        # insert the original frame
        interpolated_vif.append(current_frame)

        
        for j in range(1, num_interpolations + 1):
            frac = j / (num_interpolations + 1)
            interpolated_frame = current_frame * (1 - frac) + next_frame * frac
            interpolated_vif.append(interpolated_frame)

    # add the last frame
    interpolated_vif.append(vif_history[-1])

    return interpolated_vif



In [146]:
listings_df_VIF_new, vif_history = drop_column_using_vif_(listings_df_VIF.drop('legal_listing', axis=1), thresh=2)

c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dropping: calculated_host_listings_count (VIF: 8821.223000623602)


c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dropping: host_listings_count (VIF: 98.36273634081509)


c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


...

c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dropping: number_of_reviews_ltm (VIF: 3.178148336685998)
Dropping: availability_90 (VIF: 2.688222572599056)


c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dropping: review_scores_cleanliness (VIF: 2.467592323236401)
Dropping: reviews_per_month (VIF: 2.225361395015374)


c:\Users\SBH\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [27]:
# make the interpolated frame by linear ways
interpolated_vif_history = linear_interpolate_vif(vif_history, num_interpolations=10)

# using make_vif_gif function to generate GIF
make_vif_gif(interpolated_vif_history, num_interpolations=10, keyframe_repeats=20, interpolated_frame_repeats=1, gif_filename='interpolated_vif_evolution.gif')


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos

In [ ]:
"""%%capture --no-stdout

listings_df_VIF_new = drop_column_using_vif_(listings_df_VIF.drop('legal_listing', axis=1), thresh=2)"""

In [147]:
# After VIF now we have the 'listings_df_VIF_new'
print(f"There are {listings_df_VIF_new.shape[1]} variables after VIF operation.")

# Add legal_listing back to csv
listings_df_VIF_new['legal_listing'] = listings_df_VIF['legal_listing']


# And save the new dataframe to csv.file
listings_df_VIF_new.to_csv(os.path.join('data','yvr_listing_data_cleaned.csv'),index=False)

There are 43 variables after VIF operation.
